# Google Routes API: Single-Route Test

Tests one bicycle route fetch to validate API key, request format, and response.
Uses `routes_fetch.py` for cache-before-fetch and force-fetch support.

## Setup

In [1]:
# =============================================================================
# SETUP: Paths, load .env, get API key
# =============================================================================
from pathlib import Path
import json
import os
import sys

from dotenv import load_dotenv

load_dotenv()

cwd = Path.cwd()
project_root = cwd if (cwd / "package.json").exists() else cwd.parent.parent
prepared_dir = project_root / "prepared-data"
cache_dir = project_root / "routes-cache"

sys.path.insert(0, str(project_root / "data-pipeline"))
from routes_fetch import fetch_route

api_key = os.environ.get("GOOGLE_ROUTES_API_KEY")
if not api_key:
    raise RuntimeError(
        "Set GOOGLE_ROUTES_API_KEY in environment or .env file. "
        "Copy .env.example to .env and add your key."
    )

print("Project root:", project_root)
print("Cache dir:", cache_dir)

Project root: c:\Users\Nicol\Desktop\INF252-Course-Project
Cache dir: c:\Users\Nicol\Desktop\INF252-Course-Project\routes-cache


## Load Stations

In [2]:
# =============================================================================
# Load stations from stations.json or isochrones.json (fallback)
# =============================================================================
stations_path = prepared_dir / "stations.json"
if not stations_path.exists():
    stations_path = prepared_dir / "isochrones.json"

if not stations_path.exists():
    raise FileNotFoundError(
        f"No stations file found. Run stations_prepare.ipynb first, "
        f"or ensure isochrones.json exists in {prepared_dir}"
    )

with open(stations_path, encoding="utf-8") as f:
    data = json.load(f)

stations = data.get("data", data).get("stations", data.get("stations", []))
print(f"Loaded {len(stations)} stations from {stations_path.name}")

Loaded 292 stations from stations.json


## Fetch Single Route

In [3]:
# =============================================================================
# Fetch route: Tøyenparken (377) -> Grønlands torg (381)
# Set FORCE_ROUTES_FETCH=1 in env to bypass cache during development.
# =============================================================================
origin_id = "377"
dest_id = "381"

result = fetch_route(origin_id, dest_id, stations, api_key, cache_dir)

cached = result.get("cached", False)
resp = result.get("response", {})
routes = resp.get("routes", [])

if routes:
    r = routes[0]
    duration = r.get("duration", "N/A")
    distance_m = r.get("distanceMeters", "N/A")
    print(f"Success (cached={cached})")
    print(f"  Duration: {duration}")
    print(f"  Distance: {distance_m} m")
else:
    print(f"No route returned. Response: {resp}")

Success (cached=False)
  Duration: 184s
  Distance: 1282 m
